In [12]:
input_file = "food.jsonl"

with open(input_file) as f:
    lines = f.read().splitlines()

In [13]:
import pandas as pd
df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']

In [14]:
import json
df_inter['json_element'].apply(json.loads)

df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))

df_final.head(n=10)

,review,sentiment,category,aspects.طعم,aspects.کیفیت,aspects.ارزش غذایی,aspects.ارزش خرید,aspects.بسته بندی,aspects.ارسال
0,من یه مدته فقط از این محصول استفاده میکنم اما ...,3,حبوبات و سویا,NaN,NaN,NaN,NaN,NaN,NaN
1,اگه بخوای به صورت مکمل استفاده اش کنی خوبه وار...,1,شربت و آبمیوه,NaN,NaN,NaN,NaN,NaN,NaN
2,طعم خیلی خوبی داره و توضیحات کالا کاملا با خود...,2,تن ماهی,2,NaN,NaN,NaN,NaN,NaN
3,من این رو که گرفتم خیلی به نظر مونده بود و بشد...,-2,گوشت مرغ,-2,-2,NaN,NaN,NaN,NaN
4,کلا که نوشابه محصول مفیدی نیست. اما اونایی که ...,1,نوشابه,NaN,NaN,-1,1,NaN,NaN
5,طعم و مزه عسل رو داره من توی شگفت انگیز خریدم ...,3,عسل,1,NaN,NaN,NaN,-1,NaN
6,پسته بو داده و نمکین گلستان رو تازه استفاده کر...,1,تخمه و مغز طعم‌دار,NaN,1,NaN,NaN,NaN,NaN
7,بسیار خوش طعم و عالیه یه تن ماهی باکیفیت و خوشمزه,2,تن ماهی,2,2,NaN,NaN,NaN,NaN
8,کیفیت خوبی داره اما قیمتش بالاست. خرید میگو تا...,3,ماهی، میگو و خاویار,NaN,1,NaN,-1,NaN,NaN
9,"تاریخ روز گذشته بود, خیلی کوچک بود و تازه نبود...",-2,گوشت مرغ,NaN,-2,NaN,NaN,NaN,NaN


In [15]:
df_final.shape

(1917, 9)

In [16]:
df_final['sentiment'].value_counts()

,count
sentiment,
-2,462
2,440
1,420
-1,381
3,184
0,30


In [17]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1917 entries, 0 to 1916
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   review              1917 non-null   object
 1   sentiment           1917 non-null   object
 2   category            1917 non-null   object
 3   aspects.طعم         886 non-null    object
 4   aspects.کیفیت       553 non-null    object
 5   aspects.ارزش غذایی  89 non-null     object
 6   aspects.ارزش خرید   414 non-null    object
 7   aspects.بسته بندی   156 non-null    object
 8   aspects.ارسال       64 non-null     object
dtypes: object(9)
memory usage: 134.9+ KB


In [18]:
df_final = df_final[['review', 'sentiment']]
df_final.head(n=5)

,review,sentiment
0,من یه مدته فقط از این محصول استفاده میکنم اما ...,3
1,اگه بخوای به صورت مکمل استفاده اش کنی خوبه وار...,1
2,طعم خیلی خوبی داره و توضیحات کالا کاملا با خود...,2
3,من این رو که گرفتم خیلی به نظر مونده بود و بشد...,-2
4,کلا که نوشابه محصول مفیدی نیست. اما اونایی که ...,1


In [19]:
label_dict = {
    "-2" : "NEGATIVE",
    "-1" : "NEGATIVE",
    "0"  : "NEUTRAL",
    "1"  : "POSITIVE",
    "2"  : "POSITIVE",
    "3"  : "OTHER",
}

per_to_eng_dict = {
    "مثبت" : "POSITIVE",
    "خنثی" : "NEUTRAL",
    "منفی" : "NEGATIVE",
    "سایر" : "OTHER",
}
eng_to_per_dict = {
    "POSITIVE"  : "مثبت",
    "NEUTRAL"   : "خنثی",
    "NEGATIVE"  : "منفی",
    "OTHER"     : "سایر",
}

acceptable_classes = [
    "NEGATIVE",
    "NEUTRAL",
    "POSITIVE",
    "OTHER",
]

In [4]:
!pip install -U "transformers==4.40.0" --upgrade
!pip install accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 70.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  U

In [5]:
import transformers
import torch

model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
def get_inference(prompt):
  messages = [
    {"role": "system", "content": ".تو یک دستیار ویرایشگر متن های فارسی هستی"},
    {"role": "user", "content": """{prompt}""".format(prompt = prompt)},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=500,
      do_sample=False,
      temperature=0.5,
      top_p=0.9,
  )

  return(outputs[0]["generated_text"][len(prompt):])

In [21]:

PROMPT_FEW = """
جمله زیر نظر یک شخص است. این جمله به زبان فارسی است. بار یا احساس موجود در این جمله را شناسایی کن. \
پاسخ‌ های ممکن کلمات روبرو هستند: POSITIVE, NEUTRAL, NEGATIVE, OTHER. \
زمانی که جمله دارای احساس خاصی نیست یا شامل احساسات مختلف است از OTHER استفاده کن \
به مثالهای زیر توجه کن:

  مثال 1:
      Review: پودر قهوه دارای کیفیت مطلوب نبود وعطر و بوی در فضا پخش نمی شد
      Label: NEGATIVE


  مثال 2:
      Review: از این برند انتظار بیشتری داشتم خیلی خشک و بی مزه بود
      Label: NEGATIVE


  مثال 3:
      Review: تعداد شیر های پرچرب کمتر از کم چرب بود ۴ تا کم چرب بود و ۲ تا پر چرب
      Label: NEUTRAL


  مثال 4:
      Review: دفعه های قبل که سفارش دادم بو میداد ولی ایندفعه تازه بود و بوی بد هم نمیداد
      Label: POSITIVE


  مثال 5:
      Review: به نظرم خیلی خوشمزه بود طعم شکلات با قهوه عالی میشه
      Label: POSITIVE


  مثال 6:
      Review: از نظر کیفیت خوبه ولی از نظر قیمت فقط تو پیشنهاد ویژه خوبه
      Label: OTHER


  مثال 7:
      Review: نسبت به قیمت جدیدش قهوه های بهتری رو توی این رنج قیمت میشه خرید
      Label: NEGATIVE


  فرمت خروجی:
  فرمت خروجی شما باید یک تاپل  باشد، که در آن هر تاپل از متن نظر ورودی و برچسب احساسات لیست شده ای که تشخیص داده ای تشکیل شده باشد.

  """

In [23]:
s = 0
e = 100
part = df_final.iloc[s:e]

In [ ]:
import numpy as np

all_response = []

for index, row in part.iterrows():
    print('i: ',index)
    label = row['sentiment']
    review = row['review']

    prompt_arman = f"""
    شرح وظیفه:
    {PROMPT_FEW}

    ورودی:
    {review}
    """
    response = get_inference(prompt_arman)

    all_response.append({'review':review , 'label':label , 'response':response})
    if index%10==0 :
        np.save(f'sa_llama3_8b_arman_fewshot{s}_{e}.npy', np.array(all_response, dtype=object))


i:  0
i:  1
i:  2
i:  3
i:  4
i:  5
i:  6
i:  7
i:  8
i:  9


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


i:  10
i:  11
i:  12
i:  13
i:  14
i:  15
i:  16
i:  17
i:  18
i:  19
